In [ ]:
# --- SCRIPT 1: PREPARACIÓN DE MUESTRAS ---


import os
import random
import shutil
from google.colab import drive
from pathlib import Path


# Montar Google Drive
drive.mount('/content/drive')


# --- CONFIGURACIÓN DE RUTAS ---
# Ruta raíz de tu Drive
DRIVE_ROOT = '/content/drive/MyDrive/'
# Ruta a tu carpeta TEST (donde están las 12 especies)
TEST_DATA_PATH = os.path.join(DRIVE_ROOT, 'plantas_dataset_v2/dataset_split_anidado/test')
# Nueva carpeta donde se guardarán las 2 imágenes de muestra por especie
GRADCAM_SAMPLES_PATH = os.path.join(DRIVE_ROOT, 'GradCAM_Samples')


# Crear carpeta de muestras, eliminando la anterior si existe para empezar de cero
if os.path.exists(GRADCAM_SAMPLES_PATH):
    shutil.rmtree(GRADCAM_SAMPLES_PATH)
Path(GRADCAM_SAMPLES_PATH).mkdir(parents=True, exist_ok=True)


NUM_SAMPLES_PER_CLASS = 2 # Número de imágenes a seleccionar por especie


# --- LÓGICA DE EXTRACCIÓN ---


# Obtener nombres de las especies
species_folders = [d for d in os.listdir(TEST_DATA_PATH) if os.path.isdir(os.path.join(TEST_DATA_PATH, d))]


if not species_folders:
    print(f"ERROR: No se encontraron subcarpetas (especies) en {TEST_DATA_PATH}. Verifica la ruta.")
else:
    print(f"Iniciando selección de {NUM_SAMPLES_PER_CLASS} muestras de {len(species_folders)} especies.")


    for species_name in species_folders:
        species_dir = os.path.join(TEST_DATA_PATH, species_name)
        images = [f for f in os.listdir(species_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]


        if len(images) < NUM_SAMPLES_PER_CLASS:
            print(f"Advertencia: La especie {species_name} solo tiene {len(images)} imágenes.")
            selected_images = images
        else:
            selected_images = random.sample(images, NUM_SAMPLES_PER_CLASS)


        # Crear subcarpeta en GradCAM_Samples
        target_species_dir = os.path.join(GRADCAM_SAMPLES_PATH, species_name)
        Path(target_species_dir).mkdir(parents=True, exist_ok=True)


        # Copiar imágenes seleccionadas
        for img_name in selected_images:
            source_path = os.path.join(species_dir, img_name)
            target_path = os.path.join(target_species_dir, img_name)
            shutil.copy(source_path, target_path)


    print("\n Extracción de muestras completada.")
    print(f"Imágenes guardadas en: {GRADCAM_SAMPLES_PATH}")
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from torchcam.methods import GradCAM
from torchcam.utils import overlay_mask
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import glob


# --- CONFIGURACIÓN DE RUTAS ---
DRIVE_ROOT = '/content/drive/MyDrive/'
MODELS_DIR = os.path.join(DRIVE_ROOT, 'MODELOS')
GRADCAM_SAMPLES_PATH = os.path.join(DRIVE_ROOT, 'GradCAM_Samples')
GRADCAM_OUTPUT_PATH = os.path.join(DRIVE_ROOT, 'GRADCAM_RESULTS') # Carpeta de resultados
Path(GRADCAM_OUTPUT_PATH).mkdir(parents=True, exist_ok=True)




# --- Parámetros de Grad-CAM y modelos ---
num_classes = 12
feature_extract = False


# Mapeo de modelos y sus últimas capas convolucionales (target_layer)
MODELS_CONFIG = {
    'VGG11-BN':    {'func': models.vgg11_bn,      'folder': 'VGG11_BN',     'target_layer': 'features[-1]'}, # Última capa ReLU del último bloque
    'DenseNet-201':{'func': models.densenet201,   'folder': 'DENSENET_201', 'target_layer': 'features.denseblock4'}, # Último bloque denso
    'ResNet-101':  {'func': models.resnet101,     'folder': 'RESNET_101',   'target_layer': 'layer4'}, # Última capa de bloques
    'MobileNetV2': {'func': models.mobilenet_v2,  'folder': 'MOBILENETV2',  'target_layer': 'features[18]'}, # Última capa convolucional antes del pool
    'Inception':   {'func': models.inception_v3,  'folder': 'INCEPTION',    'target_layer': 'Mixed_7c'}, # Último bloque mixto
}


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")




# --- Funciones auxiliares de tu código ---
def set_parameter_requires_grad(model, feature_extracting):
  if feature_extracting:
    for param in model.parameters():
      param.requires_grad = False


def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
  # Tu función de inicialización modificada para este script (simplificada y ajustada para Grad-CAM)
  from torchvision import models
  model_ft = None
  input_size = 0


  config = MODELS_CONFIG.get(model_name)
  if not config:
    return None, 0


  model_func = config['func']


  # Inicializa el modelo (con pretrained=True si se desea cargar una estructura estándar)
  model_ft = model_func(weights=None) # Cargar solo la estructura sin pesos de ImageNet


  # Ajusta la capa final según tu arquitectura original
  set_parameter_requires_grad(model_ft, feature_extract)
  try:
    if model_name == "VGG11-BN":
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs, num_classes)
        input_size = 224
    elif model_name == "DenseNet-201":
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224
    elif model_name == "ResNet-101":
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224
    elif model_name == "MobileNetV2":
        num_ftrs = model_ft.classifier[1].in_features
        model_ft.classifier[1] = nn.Linear(num_ftrs, num_classes)
        input_size = 224
    elif model_name == "Inception":
        # Inception V3 requiere aux_logits=True para inicializar correctamente la estructura
        model_ft = models.inception_v3(weights=None, aux_logits=True)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.AuxLogits.fc = nn.Linear(model_ft.AuxLogits.fc.in_features, num_classes)
        model_ft.fc = nn.Linear(model_ft.fc.in_features, num_classes)
        input_size = 299
  except Exception as e:
    print(f"Error al ajustar capa final de {model_name}: {e}")
    return None, 0


  return model_ft, input_size


def preprocess_image(image_path, img_size):
  transform = transforms.Compose([
      transforms.Resize((img_size, img_size)),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  ])
  image = Image.open(image_path).convert("RGB")
  tensor = transform(image).unsqueeze(0)
  return tensor, image


# --- BUCLE PRINCIPAL DE GRAD-CAM ---


# Obtener todas las imágenes de muestra
image_paths = glob.glob(os.path.join(GRADCAM_SAMPLES_PATH, '*', '*'))
if not image_paths:
    print(f" No se encontraron imágenes en {GRADCAM_SAMPLES_PATH}. Ejecuta el Script 1 primero.")
else:
    print(f"\nIniciando Grad-CAM para {len(image_paths)} imágenes de muestra...")


    # DataFrame para resultados (opcional para un agente externo)
    gradcam_results = []


    for model_name, config in MODELS_CONFIG.items():
        print(f"\n==================================================")
        print(f"=== Generando Grad-CAM: {model_name} ===")
        print(f"==================================================")


        model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=False)
        if model_ft is None:
            continue


        # Cargar pesos entrenados del mejor modelo
        weights_path = os.path.join(MODELS_DIR, config['folder'], 'best.pt')
        try:
            checkpoint = torch.load(weights_path, map_location=device)
            model_ft.load_state_dict(checkpoint['model_state_dict'])
            print(f"Pesos cargados exitosamente desde: {weights_path}")
        except Exception as e:
            print(f" ERROR al cargar pesos para {model_name}: {e}. Saltando este modelo.")
            continue


        model_ft.eval()
        model_ft.to(device)


        # Habilitar gradientes en todas las capas para Grad-CAM (es temporal para la inferencia)
        for param in model_ft.parameters():
            param.requires_grad = True


        # Inicializar el extractor GradCAM
        try:
            cam_extractor = GradCAM(model_ft, target_layer=config['target_layer'])
        except Exception as e:
            print(f"ERROR: No se pudo inicializar GradCAM para la capa {config['target_layer']}: {e}. Saltando.")
            continue


        for i, image_path in enumerate(image_paths):
            species_name = Path(image_path).parent.name
            image_filename = Path(image_path).name


            # Preprocesar imagen y realizar forward pass
            input_tensor, original_img = preprocess_image(image_path, input_size)
            input_tensor = input_tensor.to(device)


            # Limpiar caché de gradientes por si acaso
            model_ft.zero_grad()


            out = model_ft(input_tensor)


            # Manejo específico para Inception en la inferencia
            if isinstance(out, tuple):
                out = out[0]


            pred_logits = out.cpu()
            pred_class_idx = pred_logits.argmax(dim=1).item()
            pred_confidence = torch.nn.functional.softmax(pred_logits, dim=1).max().item()


            # Asumiendo que el nombre de la carpeta (species_name) es la etiqueta correcta
            # Necesitamos mapear el índice predicho de vuelta al nombre de la clase


            # --- Tarea Pendiente: Obtener mapeo de clases para el nombre real ---
            # Si se usó ImageFolder para el entrenamiento, se necesita el mapeo idx_to_class
            # Asumo que el mapeo se obtiene si tienes acceso al DataLoader que usaste para entrenar.
            # Como no lo tenemos aquí, lo simularemos si solo tienes 12 carpetas:
            # Puedes reemplazar 'Clase Predicha' por el nombre real de la clase


            # Por ahora usaremos el índice predicho como referencia:
            predicted_class_name = f"Index_{pred_class_idx}"
            # --- Fin Tarea Pendiente ---


            # Extraer Grad-CAM map
            # cam_extractor retorna una lista de tensores (uno por capa objetivo)
            activation_map = cam_extractor(pred_class_idx, out)[0]


            # Redimensionar el mapa a las dimensiones de la imagen de entrada (Heatmap)
            activation_map = transforms.Resize(input_size)(activation_map.unsqueeze(0)).squeeze(0)


            # Superponer el mapa en la imagen original
            # Se convierte a numpy y se usa 'F' (float) para overlay_mask
            mask_np = activation_map.cpu().numpy()
            result_img = overlay_mask(original_img, Image.fromarray(mask_np, mode='F'), alpha=0.5)


            # --- Visualización y Guardado ---
            plt.figure(figsize=(10, 5))


            plt.subplot(1, 2, 1)
            plt.imshow(original_img)
            plt.title(f"Original: {species_name}")
            plt.axis('off')


            plt.subplot(1, 2, 2)
            plt.imshow(result_img)
            plt.title(f"{model_name} - Pred: {predicted_class_name} ({pred_confidence:.2f})")
            plt.axis('off')


            plt.tight_layout()


            # Guardar en la carpeta de resultados
            output_dir = os.path.join(GRADCAM_OUTPUT_PATH, model_name)
            Path(output_dir).mkdir(parents=True, exist_ok=True)
            save_filename = f"{species_name}_{Path(image_filename).stem}_GradCAM.png"
            plt.savefig(os.path.join(output_dir, save_filename), dpi=300)
            plt.close() # Cierra la figura para evitar sobrecarga de memoria


            # Acumular resultados para el agente
            gradcam_results.append({
                'Model': model_name,
                'Species': species_name,
                'Filename': image_filename,
                'Predicted_Index': pred_class_idx,
                'Confidence': pred_confidence,
                'GradCAM_Path': os.path.join(output_dir, save_filename)
            })


            if (i + 1) % 10 == 0:
                print(f"   Procesadas {i+1} de {len(image_paths)} imágenes para {model_name}.")


    print("\n Generación de Grad-CAMs completada.")
    print(f"Resultados guardados en la carpeta: {GRADCAM_OUTPUT_PATH} (una subcarpeta por modelo).")


    # Exportar Dataframe de resultados para el agente
    final_gradcam_df = pd.DataFrame(gradcam_results)
    final_gradcam_df.to_csv(os.path.join(GRADCAM_OUTPUT_PATH, 'GradCAM_Index_Results.csv'), index=False)


    # Puedes agregar este DataFrame a tu agente de Grad-CAM para que analice los patrones.
